In [1]:
import IPython.display as ipd
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
import librosa.display

In [3]:
def extract(input_df,sound):
    input_df['end']=input_df['duration'].cumsum()
    input_df['start'] = input_df['end'] - input_df['duration']
    input_df['sig_start'] = (input_df['start']*22050).astype(int)
    input_df['sig_end'] = (input_df['end']*22050).astype(int)
    data = input_df[['recording','speaker','word','duration','end','start','sig_start','sig_end']]

    
    word_id = []
    ID = 0
    for i in range(len(data['word'].values)):
        words = data['word'].values
        if i == 0:
            insert = str(data.iloc[i,:]["recording"]) + '_' + str(words[i]) + '_' + str(ID)
            word_id.append(insert)
        else:
            if words[i] == words[(i-1)]:
                insert = str(data.iloc[i,:]["recording"]) + '_' + str(words[i]) + '_' + str(ID)
                word_id.append(insert)
            else:
                ID += 1
                insert = str(data.iloc[i,:]["recording"]) + '_' + str(words[i]) + '_' + str(ID)
                word_id.append(insert)

    data['word_id'] = word_id


    for i in range(len((data['word_id'].unique()))):
        df = data[data['word_id'] == data['word_id'].unique()[i]].copy()
        end = df['sig_end'].max()
        start = df['sig_start'].min()
        df1 = df.iloc[0,:]
        df1['start'] = start
        df1['end'] = end
        df1['duration'] = df1['end']-df1['start']
        if i == 0:
            df2 = df1.copy()
        elif i > 0:
            df2 = pd.concat([df2,df1],axis = 1)
    df2=df2.T[['recording', 'speaker', 'word','word_id', 'duration', 'start', 'end']]
    
    word_sound = []
    for i in range(len(df2)):
        begin,end = df2['start'].values[i],df2['end'].values[i]
        word = sound[begin:end]
        word_sound.append(word)
    

    
    return df2,word_sound

In [4]:
def speaker_words(Letters,rate=22050):
    empty = pd.DataFrame(columns = ['recording','speaker','word','word_id','duration','start','end'])
    spoken = []
    
    # Extraction Step
    for Letter in Letters:
        df = pd.read_csv('UK\\Tabular\\{Letter}.csv'.format(Letter=Letter))

        
        recordings = df['recording'].unique()
        
        iteration = 0
        for recording in recordings:
            df1 = df[df['recording'] == recording]
            
            # Only A has subfolders
            if Letter == 'A':
                sig, sr = librosa.load('UK\\Sounds\\{Letter}\\{recording}\\{recording}.wav'.format(Letter=Letter,recording=recording)
                                       , sr=rate)
            else:
                sig, sr = librosa.load('UK\\Sounds\\{Letter}\\{recording}.wav'.format(Letter=Letter,recording=recording)
                                       , sr=rate)
            
            df3,s = extract(df1,sig)
            
            spoken.append(s)
            
            # DataFrame Creation
            if iteration == 0 :
                df4 = df3
                iteration += 1
            else:
                df4 = pd.concat([df4,df3],axis = 0)
                iteration += 1
            print(Letter,iteration)
        empty = pd.concat([empty,df4],axis=0)
    print('Extraction completed')
    return empty,spoken
                
                

In [6]:
Letters = ['A','B','C','D','E','F','G','H','J','K','M']
final_df,spoken = speaker_words(Letters,rate=22050)

C:\Users\geral\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\geral\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\geral\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

A 1
A 2
A 3
A 4
A 5
A 6
A 7
A 8
A 9
A 10
A 11
A 12
B 1
B 2
B 3
B 4
C 1
D 1
D 2
D 3
E 1
E 2
F 1
F 2
F 3
F 4
G 1
G 2
G 3
G 4
G 5
H 1
H 2
H 3
H 4
H 5
J 1
J 2
J 3
J 4
J 5
J 6
K 1
K 2
M 1
M 2
M 3
M 4
M 5
M 6
M 7
M 8
M 9
Extraction completed


In [7]:
final_df = final_df.reset_index(drop = True)
# final_df = final_df[final_df['duration'] > 11000]
final_df = final_df[~final_df['word'].isin(['#','END'])]
final_df = final_df[final_df['word'].notnull()]
final_df = final_df[final_df['word'].str.isalpha()]
final_df = final_df.reset_index()
final_df

,index,recording,speaker,word,word_id,duration,start,end
0,1,A01,F1,Good,A01_Good_1,2425,12546,14971
1,2,A01,F1,morning,A01_morning_2,8600,14971,23571
2,4,A01,F1,more,A01_more_4,3528,39226,42754
3,5,A01,F1,news,A01_news_5,7277,42754,50031
4,6,A01,F1,about,A01_about_6,4630,50031,54661
...,...,...,...,...,...,...,...,...
51428,62661,M09,M50,Burns,M09_Burns_495,9261,3033462,3042723
51429,62662,M09,M50,night,M09_night_496,5513,3042723,3048236
51430,62664,M09,M50,here,M09_here_498,4851,3057805,3062656
51431,62665,M09,M50,on,M09_on_499,4631,3062656,3067287


In [16]:
USA_words = pd.read_csv('USA_words.csv')
USA_words.columns = ['idx','word']
US_List = list(USA_words['word'].str.upper().unique())
final_df['word'] = final_df['word'].str.upper()

In [52]:
final_df.to_csv('df.csv')

In [19]:
intersection_df = final_df[(final_df['word'].isin(US_List)) & (final_df['duration'] > (22050/2))]
intersection_df

,index,recording,speaker,word,word_id,duration,start,end
33,41,A01,F1,UNIVERSITY,A01_University_41,13671,259506,273177
42,53,A01,F1,ANNOUNCING,A01_announcing_53,12568,342811,355379
52,64,A01,F1,UNIVERSITY,A01_university_64,14994,397649,412643
102,128,A01,F1,MINORITY,A01_minority_128,11466,833688,845154
205,255,A01,F1,HIMSELF,A01_himself_255,11466,1604865,1616331
...,...,...,...,...,...,...,...,...
51153,62342,M09,F18,WAITING,M09_waiting_176,11686,1047948,1059634
51188,62384,M09,M50,ELEVEN,M09_eleven_218,11466,1265758,1277224
51238,62437,M09,M50,SUPPLIES,M09_supplies_271,11466,1604732,1616198
51268,62470,M09,M50,SELECTION,M09_selection_304,12127,1822234,1834361


In [27]:
spoken_words = []
for i in spoken:
    for x in i:
        spoken_words.append(x)

In [51]:
from scipy.io.wavfile import write
for idx,word_id in zip(intersection_df['index'],intersection_df['word_id']):
    
    write('Intersection_wav\\{word_id}.wav'.format(word_id=word_id), 22050,data=spoken_words[idx])

In [ ]:
# The remaining cells are from my attempt to somehow put all the sound arrays into one dataframe

In [ ]:
from scipy.sparse import csr_matrix
sound_matrix = csr_matrix((len(final_df), longest), dtype=np.float64).toarray()
for i in range(len(spoken_words)):
    sound_matrix[i,0:len(spoken_words[i])] = spoken_words[i]

In [48]:
test = np.genfromtxt('Intersection\\A01_announcing_53.csv',delimiter=',',skip_header=1)

In [49]:
# test = np.array(test)
from scipy.io.wavfile import write
write('test.wav', 22050,data=test)

In [50]:
ipd.Audio('test.wav')